In [1]:
# 지금까지 배운내용을 기초로 tensorflow의 mnist example을 
# cnn으로 구현해보아요

In [ ]:
# module 삽입

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd

In [3]:
# 데이터 불러오기

mnist = input_data.read_data_sets("./data/mnist", one_hot = True)

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [4]:
# 데이터 전처리

# 결측치, 이상치, 정규화, feature engineering
# tf 제공 mnist 는 이미 전처리가 진행된 상태

#### Model 정의: tensorflow graph 생성

In [5]:
# 그래프 초기화

tf.reset_default_graph()

In [6]:
# placeholder

X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
# keep_rate = tf.placeholder(tf.float32)
drop_rate = tf.placeholder(tf.float32)

#### Convolutional Neural Network 과정

In [7]:
# image data 설정

x_img = tf.reshape(X,[-1,28,28,1])

# 입력 데이터 X를 4차원으로 변환
# 해당 데이터는 흑백이기에 channel 1
# 해당 데이터의 갯수는 -1

In [9]:
# convolution 과정

# 3*3*1 : 크기와 depth, 32 : channel 갯수
'''F1 = tf.Variable(tf.random_normal([3,3,1,32]), name='filter1')
L1 = tf.nn.conv2d(x_img,
                  F1,
                  strides = [1,1,1,1],
                  padding = "SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,
                    ksize = [1,2,2,1],
                    strides = [1,2,2,1],
                    padding = "SAME")'''

# layer 패키지 사용한 다른 방법
# 필터를 따로 정의하지 않는 방법도 있음 : 하나의 식으로 표현
# 데이터, 필터수, 필터사이즈, 패딩, 스트라이드, 이후의 작업
# 단, pooling 작업은 따로 해줘야 되고 layers 패키지 사용하면서 다른 함수 사용
L1 = tf.layers.conv2d(inputs = x_img,
                      filters = 32,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1,
                      activation = tf.nn.relu)
L1 = tf.layers.max_pooling2d(inputs = L1,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)
# 추가적으로 convolution 수행할 때 dropout 실행할 수 있음
# 원래는 FC에서 실행하지면 시간을 더 줄이기 위해서 현재 과정에서 사용 가능
# 이때는 rate 변수 사용
L1 = tf.layers.dropout(inputs = L1, 
                       rate = drop_rate)




L2 = tf.layers.conv2d(inputs = L1,
                      filters = 64,
                      kernel_size = [3,3],
                      padding ="SAME",
                      strides = 1,
                      activation = tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs = L2,
                             pool_size = [2,2],
                             padding = "SAME",
                             strides = 2)
print(L2.shape)

(?, 7, 7, 64)


#### Dense layer, Fully Connected layer 과정

In [10]:
# data 준비

convoled_data = tf.reshape(L2, [-1,7*7*64])

In [12]:
# W, b, layer

'''W1 = tf.get_variable("weight1", 
                     shape = [7*7*64, 256],
                     initializer = tf.contrib.layers.xavier_initializer_conv2d())
b1 = tf.Variable(tf.random_normal(shape=[256]),
                 name = 'bias1')
_layer1 = tf.nn.relu(tf.matmul(convoled_data, W1) + b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_rate)'''

# layer 패키지에서 제공해주는 함수 사용하면 더욱 간편하게 서술 가능
# W, b에 대한 곱을 비롯한 모든 계산을 알아서 해줌
layer1 = tf.layers.dense(inputs = convoled_data,
                        units = 256,   #출력값
                        activation = tf.nn.relu)

# dropout 또 진행해도 좋음
layer1 = tf.layers.dropout(inputs = layer1,
                           rate = drop_rate)


layer2 = tf.layers.dense(inputs = layer1,
                        units = 128,   
                        activation = tf.nn.relu)
layer2 = tf.layers.dropout(inputs = layer2,
                          rate = drop_rate)


layer3 = tf.layers.dense(inputs = layer2,
                        units = 512,  
                        activation = tf.nn.relu)
layer3 = tf.layers.dropout(inputs = layer2,
                          rate = drop_rate)


In [13]:
# H

'''logit = tf.matmul(layer2, W3) + b3
H = tf.nn.softmax(logit)'''
# softmax 내가 적용하지 않아도 내부에서 모두 적용

H = tf.layers.dense(inputs = layer3, units=10)

In [14]:
# cost

'''cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
                                                                 labels = Y))'''
cost = tf.losses.softmax_cross_entropy(Y, H)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [15]:
# train

train = tf.train.AdadeltaOptimizer(learning_rate = 0.01).minimize(cost)

In [16]:
# session

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# 결국 우리의 mnist 에제는 multinomial 예제
# 이미지 1개에 대한 예측값, H 도출값은 [0.5, 0.3, 0.2, 0.001, 0.99, 0.44 ...]

# xavier를 사용한다고 하더라도 결론적으로 w, b의 값은 초기에 랜덤으로 잡힘
# 즉, 학습을 새로 시킬때마다 학습의 결과가 매번 다르게 나올 수 밖에 없음

# ensenble은 모델을 여러개 만들고 (10개의 모델)
# 이미지 1개에 대한 각 모델의 예측값을 구한다.
# 모델1: [0.5, 0.3, 0.2, 0.001, 0.99, 0.44 ...]
# 모델2: [0.4, 0.3, 0.11, 0.05, 0.90, 0.64 ...] ...

# 이 모든 모델의 값을 모두 더하여 prediction을 한다.
# sun = [0.9, 0.6, ... ] 

# 프로그램을 사용하는 것이 아닌 logic으로 해결해야 함